# Transposition Cipher

Transposition cipher is a cipher technique where we shuffle the characters position of plain text in blocks. For example, if we choose a key length of 5, and decide the key as -

(3, 4, 1, 5, 2)

Then, on each block of 5 in the plain text, 1st character goes to position 3, 2nd to 4 and so on.

## Implementation

Firstly we define an encoding function that will be used to take plaintext and convert it to a 26 character encoding. (By converting all letters to upper case and discarding all remaining characters). We will also add padding character Z to make the length a multiple of M.

We also define M = 5 which will be the key length.

In [64]:
# key length
M = 5

In [65]:
def encode(string):
    result = ''
    for letter in string:
        if letter.isalpha():
            result += letter.upper()
    padding_length = (M - len(result)) % M
    return result + padding_length * 'Z'

# remove padding characters
def decode(string):
    return string.rstrip('Z')

Lets declare a plain text that we would need to encrypt.

In [66]:
P = 'Enemy might attack tonight. Stay alert!'

Encoding this string, we get -

In [67]:
T = encode(P)
print(T)

ENEMYMIGHTATTACKTONIGHTSTAYALERTZZZ


Since we will be only having 26 characters, we declare Zp as closed ring of 26 integers.

In [68]:
Zp = Integers(26)
print(Zp)

Ring of integers modulo 26


We now declare a key which is a permutation of numbers from 1 to M represented as sequence of characters.

In [69]:
characters = [chr(i + ord('A')) for i in range(M)]
key = ''
while len(characters) > 0:
    i = randint(0, len(characters) - 1)
    key += characters[i]
    del characters[i]

In [70]:
print('Key -', key)

Key - BCDAE


In [82]:
def transpositioncipher(text, cipher_key):
    cipher = ''
    for i in range(0, len(text), M):
        # iterate over each block
        block = text[i:i+M]
        new_block = ['A' for _ in range(M)]
        for j in range(M):
            k = ord(cipher_key[j]) - ord('A')
            new_block[j] = block[k]
        for c in new_block:
            cipher += c
    return cipher
    

def transpositiondecipher(cipher_text, cipher_key):
    text = ''
    for i in range(0, len(cipher_text), M):
        # iterate over each block
        block = cipher_text[i:i+M]
        new_block = ['A' for _ in range(M)]
        for j in range(M):
            k = ord(cipher_key[j]) - ord('A')
            new_block[k] = block[j]
        for c in new_block:
            text += c
    return text
    

Now we test the cipher and decipher algorithms by encrypting and decrypting the text P

In [83]:
T = encode(P)
C = transpositioncipher(T, key)
D = transpositiondecipher(C, key)
D_stripped = decode(D)
print(f'Given text - "{P}"')
print(f'Encoded - {T}')
print(f'Key - {key}')
print(f'Cipher text - {C}')
print(f'Decipher text - {D}')
print(f'Stripped text - {D_stripped}')

Given text - "Enemy might attack tonight. Stay alert!"
Encoded - ENEMYMIGHTATTACKTONIGHTSTAYALERTZZZ
Key - BCDAE
Cipher text - NEMEYIGHMTTTAACTONKIHTSGTYALAETZZRZ
Decipher text - ENEMYMIGHTATTACKTONIGHTSTAYALERTZZZ
Stripped text - ENEMYMIGHTATTACKTONIGHTSTAYALERT


## Test Against Builtin Cipher

Now, we can test the result against the built in Hill Cipher in sagemath.

In [91]:
A = TranspositionCryptosystem(AlphabeticStrings(), M)
E = A.encoding(encode(P))
# sagemath cryptosystem expects key as an array
K = [ord(c) - ord('A') + 1 for c in key]
print(f'Text - {P}')
print(f'Encoded - {E}')
print(f'Key -\b{key}')
C_test = A.enciphering(K, E)

# calculate the inverse key for K
Ki = [1] * len(K)
for i in range(len(K)):
    Ki[K[i] - 1] = i + 1
print(Ki)

# encipher with inverse key is same as decipher with key
D_test = A.enciphering(Ki, C_test)
# convert to python string
C_test = str(C_test)
D_test = str(D_test)
D_test_stripped = decode(D_test)
print(f'Cipher text - {C_test}')
print(f'Decipher text - {D_test}')
print(f'Stripped text - {D_test_stripped}')

Text - Enemy might attack tonight. Stay alert!
Encoded - ENEMYMIGHTATTACKTONIGHTSTAYALERTZZZ
Key -BCDAE
[4, 1, 2, 3, 5]
Cipher text - NEMEYIGHMTTTAACTONKIHTSGTYALAETZZRZ
Decipher text - ENEMYMIGHTATTACKTONIGHTSTAYALERTZZZ
Stripped text - ENEMYMIGHTATTACKTONIGHTSTAYALERT


Comparing the built in cipher result with our implementation -

In [92]:
print('Results \t Implementation \t Built-in')
print('-' * 80)
print(f'Cipher Text \t {C} \t {C_test}')
print(f'Decipher Text \t {D} \t {D_test}\n')
if C_test == C and D_test == D:
    print('Implementation is CORRECT')
else:
    print('Implementatiokn is INCORRECT')

Results 	 Implementation 	 Built-in
--------------------------------------------------------------------------------
Cipher Text 	 NEMEYIGHMTTTAACTONKIHTSGTYALAETZZRZ 	 NEMEYIGHMTTTAACTONKIHTSGTYALAETZZRZ
Decipher Text 	 ENEMYMIGHTATTACKTONIGHTSTAYALERTZZZ 	 ENEMYMIGHTATTACKTONIGHTSTAYALERTZZZ

Implementation is CORRECT


## Cryptoanalysis

### Brute Force

Since the key domain for shift cipher, a brute force attack is very easy to carry out, making this cipher very weak.

In [11]:
print('Total keys -', len(k_domain))

Total keys - 26


We must now get a list of english words that can be used to detect existence of english words in our bruteforced decipher text.

A good list of 3000 most used english words is here -
https://github.com/aneeshsharma/EnglishWords/raw/main/common3000.txt

We download the list of words and convert it to a list and then convert all words into upper case.

In [12]:
import requests
url = 'https://github.com/aneeshsharma/EnglishWords/raw/main/common3000.txt'

words_file = requests.get(url, allow_redirects=True)
words_file_obj = open('words.txt', 'wb')
words_file_obj.write(words_file.content)
words_file_obj.close()

In [13]:
words = open('words.txt').read().split()
words = [word.upper() for word in words]

In [14]:
print(f'Number of words in dictionary - {len(words)}')

Number of words in dictionary - 3000


A function can be defined to find all substrings in a string that are among the 3000 most common english words. This can give us a measure of the likelihood of the string being an english sentance.

In [15]:
# function to find english words in a string according to word list
def find_words(string):
    l = len(string)
    found = []
    for i in range(l):
        for j in range(i, l):
            word = string[i:j+1]
            if len(word) <= 1:
                continue
            if word in words:
                found.append(string[i:j+1])
    return found

Now, we must try to decipher the encrypted text using the list of keys we have and try to compare and count any english words found in the text. More the words detected, more likely is it that the key is correct.

In [16]:
keys = {}
max_words = 0
for candidate in k_domain:
    candidate_text = shiftdecipher(C, candidate)
    found = find_words(candidate_text)
    if len(found) > 3:
        if len(found) > max_words:
            max_words = len(found)
        keys[candidate] = len(found)

print('Key \t\t Likelihood')
for likely_key in keys:
    print(f'{likely_key} \t\t {keys[likely_key]}')

Key 		 Likelihood
9 		 4
17 		 4
24 		 10


Now that we have a list of keys and their likelihood of being correct, we can display the keys and the possible plain text that are the most likely to be correct.

In [17]:
text_list = [[] for _ in range(max_words + 1)]
for likely_key in keys:
    count = keys[likely_key]
    text_list[count].append(shiftdecipher(C, likely_key))

print('Most likely strings -')
for text in text_list[max_words]:
    print(f'{text}')

Most likely strings -
ENEMYCANATTACKTONIGHTSTAYALERT


### Statistical Mono

We get the frequency of all alphabets in English language. The file at - https://github.com/aneeshsharma/EnglishWords/blob/main/character_frequency.json is a json file with frequency of all letters in English language.

In [18]:
import requests
url = 'https://github.com/aneeshsharma/EnglishWords/raw/main/character_frequency.json'

characters_list = requests.get(url, allow_redirects=True)
characters_file_obj = open('characters.json', 'wb')
characters_file_obj.write(characters_list.content)
characters_file_obj.close()

In [19]:
def dict_to_sorted_tuple(d):
    d = [(key, d[key]) for key in d]
    return sorted(d, key=lambda item: -item[1])

In [20]:
import json
frequencies_file = json.loads(open('characters.json', 'r').read())
frequencies = dict_to_sorted_tuple(frequencies_file)

In [21]:
frequencies

[('E', 12.6),
 ('T', 9.37),
 ('A', 8.34),
 ('O', 7.7),
 ('N', 6.8),
 ('I', 6.71),
 ('H', 6.11),
 ('S', 6.11),
 ('R', 5.68),
 ('L', 4.24),
 ('D', 4.14),
 ('U', 2.85),
 ('C', 2.73),
 ('M', 2.53),
 ('W', 2.34),
 ('Y', 2.04),
 ('F', 2.03),
 ('G', 1.92),
 ('P', 1.66),
 ('B', 1.54),
 ('V', 1.06),
 ('K', 0.87),
 ('J', 0.23),
 ('X', 0.2),
 ('Q', 0.09),
 ('Z', 0.06)]

Now, we try to map the most frequent character in the cipher text to most frequent english letter and check, if not we try with the next most frequent enlglish letter and so on.

In [22]:
def frequency(string):
    f = {}
    for x in string:
        if x in f:
            f[x] += 1
        else:
            f[x] = 1
    return f

In [23]:
C_freq = dict_to_sorted_tuple(frequency(C))

In [24]:
statistic_keys = {}
max_words = 0
for freq in frequencies:
    letter = freq[0]
    candidate = (ord(C_freq[0][0]) - ord(letter)) % 26
    candidate_text = shiftdecipher(C, candidate)
    found = find_words(candidate_text)
    if len(found) > 3:
        if len(found) > max_words:
            max_words = len(found)
        statistic_keys[candidate] = len(found)

print('Key \t\t Likelihood')
for likely_key in statistic_keys:
    print(f'{likely_key} \t\t {keys[likely_key]}')

Key 		 Likelihood
24 		 10
17 		 4
9 		 4


In [25]:
text_list = [[] for _ in range(max_words + 1)]
for likely_key in statistic_keys:
    count = keys[likely_key]
    text_list[count].append(shiftdecipher(C, likely_key))

print('Most likely strings -')
for text in text_list[max_words]:
    print(f'{text}')

Most likely strings -
ENEMYCANATTACKTONIGHTSTAYALERT
